In [ ]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv')

In [ ]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [ ]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [ ]:
df['fever'].value_counts

<bound method IndexOpsMixin.value_counts of 0     103.0
1     100.0
2     101.0
3      98.0
4     101.0
      ...  
95    104.0
96    101.0
97    101.0
98     98.0
99     98.0
Name: fever, Length: 100, dtype: float64>

In [ ]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [ ]:
X_train

,age,gender,fever,cough,city
71,75,Female,104.0,Strong,Delhi
59,6,Female,104.0,Mild,Kolkata
81,65,Male,99.0,Mild,Delhi
84,69,Female,98.0,Strong,Mumbai
44,20,Male,102.0,Strong,Delhi
...,...,...,...,...,...
70,68,Female,101.0,Strong,Delhi
53,83,Male,98.0,Mild,Delhi
17,40,Female,98.0,Strong,Delhi
5,84,Female,NaN,Mild,Bangalore


In [ ]:
#Adding simple imputer to fill fever null values
si = SimpleImputer()

In [ ]:
X_train_fever = si.fit_transform(X_train[['fever']])

In [ ]:
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [ ]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [ ]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [ ]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed

array([[ 75.        , 104.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [  6.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 65.        ,  99.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 69.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 20.        , 102.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 23.        , 100.66197183,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 23.        ,  98.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 26.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 64.        , 102.        ,   1.        ,   0.        ,
          0.    

In [ ]:
#with column transformer
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers = [('tnf1',SimpleImputer(),['fever']),('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [ ]:
transformer.fit_transform(X_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   6.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  65.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [100.66197183,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.    

In [ ]:
transformer.fit_transform(X_train).shape

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [ ]:
transformer.transform(X_test).shape

(20, 7)